In [2]:
#https://rapidapi.com/apidojo/api/yahoo-finance1?endpoint=apiendpoint_f6ed13cb-defc-468f-a6a0-ee4ad28bf8af
import yahoo_fin.stock_info as si
import pandas as pd
import math as math

In [13]:
#Nasdaq: tickers_nasdaq()
#S&P500: tickers_sp500()
#Others: tickers_other()
nasdaq = si.tickers_nasdaq()

In [3]:
import requests
import json
def callApi(url, stock):

    querystring = {"symbol": stock }

    headers = {
        'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
        'x-rapidapi-key': "b057006049msh2dd6f48e0283787p151e0djsnf76cd322583c"
        }
    try:
        response = requests.request("GET", url, headers=headers, params=querystring)
        balance_sheet = json.loads(response.text)
        return balance_sheet
    except ValueError:
        print("Unknown expeption on" + stock)
        return {}

In [49]:
summary_result = {}
def totalDebt(stock):
    global summary_result
    summary_result = callApi("https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-summary", stock)
    if not("financialData" in summary_result.keys()):
        return 0
    if ("totalDebt" in summary_result["financialData"].keys()) and ("raw" in summary_result["financialData"]["totalDebt"].keys()):
        return summary_result["financialData"]["totalDebt"]["raw"]
    else:
        return 0


def priceToBook(stock):
    global summary_result
    #summary_result = callApi("https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-summary", stock)
    if not("defaultKeyStatistics" in summary_result.keys()):
        return 0
    if ("priceToBook" in summary_result["defaultKeyStatistics"].keys()) and ("raw" in summary_result["defaultKeyStatistics"]["priceToBook"].keys()):
        return summary_result["defaultKeyStatistics"]["priceToBook"]["raw"]
    else:
        return 0

def peValue(stock):
    #summary_result = callApi("https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-summary", stock)
    trailingPE = {}
    forwardPE = {}
    global summary_result
    if not("summaryDetail" in summary_result.keys()):
        return {"trailingPE": 0.0, "forwardPE":0.0}  
    if ("trailingPE" in summary_result["summaryDetail"].keys()) and ("raw" in summary_result["summaryDetail"]["trailingPE"].keys()):
        trailingPE = summary_result["summaryDetail"]["trailingPE"]["raw"]
    else: 
        trailingPE = 0.0
    if "forwardPE" in summary_result["summaryDetail"].keys() and "raw" in summary_result["summaryDetail"]["forwardPE"].keys():
        forwardPE = summary_result["summaryDetail"]["forwardPE"]["raw"]
    else:
        forwardPE = 0.0
    return {"trailingPE": trailingPE, "forwardPE":forwardPE} 

In [45]:
def equity(stock):
    global summary_result
    summary_result = callApi("https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-balance-sheet", stock)
    if not("balanceSheetHistoryQuarterly" in summary_result.keys()):
        return 0
    if ("balanceSheetStatements" in summary_result["balanceSheetHistoryQuarterly"].keys()) and  len(summary_result["balanceSheetHistoryQuarterly"]["balanceSheetStatements"]) > 0:
        balanceSheetLastStatements = summary_result["balanceSheetHistoryQuarterly"]["balanceSheetStatements"][0]
        if ("totalStockholderEquity" in balanceSheetLastStatements.keys()) and  ("raw" in balanceSheetLastStatements["totalStockholderEquity"].keys()):
            return balanceSheetLastStatements["totalStockholderEquity"]["raw"]
    else:
        return 0

In [53]:

tickers = ['LUV','TELA', 'FB', 'MAR', 'JPM','HTZ', 'DAL' ]
dataSet = [] 
for ticker in tickers:
    data = {}
    data["Ticker"] = ticker
    #
    #      Dept
    #
    
    # Dept to Equity Ratio
    total_debt = totalDebt(ticker)
    toatl_equity = equity(ticker)
    debt_equity_ratio = total_debt / toatl_equity
   
    data["DebtToEquity < 0.5"] = debt_equity_ratio
    if debt_equity_ratio < 0.50 and toatl_equity != 0 :
        data["Good D\E"] = True
    else: 
        data["Good D\E"] = False
    
    #
    #      BASIC Look
    #

    price_Book = priceToBook(ticker)
    pe_Value = peValue(ticker)
    data["Price_Book < 1.5"] = price_Book
    data["TrailingPE < 15"] = pe_Value["trailingPE"]
   # data["ForwardPE < 15"] = pe_Value["forwardPE"]
    validate = 0.0
    
    if pe_Value['trailingPE'] != 0:
        validate = pe_Value['trailingPE'] * price_Book
        data["Trailing Value < 22.5"] = validate
        if validate < 22.5 and price_Book != 0:
            data["Trailing Look"] = True
        else: 
            data["Trailing Look"] = False 
    else:
        data["Trailing Value < 22.5"] = 0
        data["Trailing Look"] = False
        
#    if pe_Value['forwardPE'] != 0 and price_Book != 0:
#        validate = pe_Value['forwardPE'] * price_Book
#        data["Forward Value < 22.5"] = validate
#        if validate < 22.5:
#            data["Forward Look"] = True
#        else: 
#            data["Forward Look"] = False
#    else:
#        data["Forward Value < 22.5"] = 0
#       data["Forward Look"] = False
    dataSet.append(data.copy())
pd.DataFrame(dataSet)

,Ticker,DebtToEquity < 0.5,Good D\E,Price_Book < 1.5,TrailingPE < 15,Trailing Value < 22.5,Trailing Look
0,LUV,1.056812,False,0,128.426220,0.0,False
1,TELA,0.481428,True,0,0.000000,0.0,False
2,FB,0.099631,True,0,36.989483,0.0,False
3,MAR,-161.632917,True,0,74.246086,0.0,False
4,JPM,2.946613,False,0,13.609587,0.0,False
5,HTZ,26.246869,False,0,0.000000,0.0,False
6,DAL,3.956732,False,0,0.000000,0.0,False
